### Part I: SORT raw images by channel

In [ ]:
import os
import shutil
import glob

os.chdir('/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/remnantplate/') 

# Set your prefix
prefix = 'rem_test'  # Replace this with your current prefix

# Step 1: List all TIF files matching the pattern
files = glob.glob(f"{prefix}_Plate_R_*.TIF")


# Step 2: Extract the 'd0', 'd1', or 'd2' from the filenames
prefixes = set()
for file in files:
    prematch = file.split('.')[0]  # Extract the relevant part like 'd0', 'd1', 'd2'
    match = prematch[-2:]
  
    if match.startswith('d') and match[1] in '012':
        prefixes.add(match)

# Step 3: For each unique prefix, create a directory and move the matching files
for prefix_match in sorted(prefixes):
    
    # Create directory if it doesn't exist
    os.makedirs(prefix_match, exist_ok=True)
    
    # Move matching files into the respective directory
    for file in files:
        if prefix_match in file:
            shutil.move(file, os.path.join(prefix_match, os.path.basename(file)))

print("Files have been organized.")


Part II: use CELLPOSE to identify GFP positive cells

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io, utils
from cellpose.io import imread
import os

modelpath = "/Users/Alexander.Morano/Desktop/cellpose_training/10x training/models/10X_NewCyto"
filepath = "/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/remnantplate/d0/"

os.chdir(filepath)
files = os.listdir()
print(len(files))

model = models.CellposeModel(gpu = False, pretrained_model = modelpath)
channels = [[0,0]]

for i in files:
    image = io.imread(i)
    masks, flows, styles = model.eval(image, diameter = 60.0, channels = channels)
    io.save_rois(masks, i)

### Part III: Rename and reoganize resulting files.

In [ ]:
import os

fileLocation = "/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/titration/d2/"
fileList = os.listdir(fileLocation)
print(fileList)

for ii in fileList:
    newName = ii.replace('d2','d0_rois')
    if newName != ii:
        os.rename(fileLocation+ii,fileLocation+newName)
print(fileList)

In [ ]:
import glob
import shutil

source_files = "/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/titration/d0/*.zip"
target_folder = "/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/titration/d2/"

# retrieve file list
filelist = glob.glob(source_files)
for single_file in filelist:
     # move file with full paths as shutil.move() parameters
    shutil.move(single_file,target_folder) 

### Part IV: After Running FIJI macro on output of 2+3; sort by well.

In [ ]:
import os
import shutil
import glob

os.chdir("/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/remnantplate/results/") 

# Set your prefix
prefix = 'rem_test'  
# Replace this with your current prefix

# Step 1: List all files matching the pattern
files = glob.glob(f"{prefix}_Plate_R_p00_0_*")

# Step 2: Extract the well name from this title.
prefixes = set()
for file in files:
    prematch = file.split('_')[6]
    
    match = prematch[:3]
    print(match)
  
    if match[0] in ('ABCDEFGH') and match[2] in '0123456789':
        prefixes.add(match)

# Step 3: For each unique prefix, create a directory and move the matching files
for prefix_match in sorted(prefixes):
    
    # Create directory if it doesn't exist
    os.makedirs(prefix_match, exist_ok=True)
    
    # Move matching files into the respective directory
    for file in files:
        if prefix_match in file:
            shutil.move(file, os.path.join(prefix_match, os.path.basename(file)))

print("Files have been organized AGAIN.")


Part V: Combine CSVs and produce output CSVs and graphs

In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt

root_path = "/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/remnantplate/results/"

for root, sub, files in os.walk(root_path):
    filenames = [os.path.join(root, filename) for filename in files 
                 if filename.endswith('.csv')]
    
    flist = []
    plt.clf()
    for filename in filenames:
        print(os.path.join(root, filename))
        df = pd.read_csv(filename)
        flist.append(df)
        df_out = pd.concat(flist)
        df_out.loc['mean of cells'] = df_out.mean()
        df_out.to_csv(os.path.join(root, 'combined.csv'))
        fig = df_out['Mean'].hist(bins=[0, 100, 500, 1000, 2000, 3000, 4000])
        plt.savefig(os.path.join(root, 'fig'))
      

Part VI: add M images to the same subdirectories

In [ ]:
import os
import shutil
import glob

os.chdir("/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/remnantplate/results/") 

# Set your prefix
prefix = 'rem_test'  # Replace this with your current prefix

# Step 1: List all files matching the pattern
files = glob.glob(f"{prefix}_Plate_M_p00_0_*")

# Step 2: Extract the well name from this title.
prefixes = set()
for file in files:
    prematch = file.split('_')[6]
    
    match = prematch[:3]
    print(match)
  
    if match[0] in ('ABCDEFGH') and match[2] in '0123456789':
        prefixes.add(match)

# Step 3: For each unique prefix, create a directory and move the matching files
for prefix_match in sorted(prefixes):
    
    # Create directory if it doesn't exist
    os.makedirs(prefix_match, exist_ok=True)
    
    # Move matching files into the respective directory
    for file in files:
        if prefix_match in file:
            shutil.move(file, os.path.join(prefix_match, os.path.basename(file)))

print("Files have been organized AGAIN.")

Part VII: Finally, you can generate a csv telling you which wells are positive and which are negative!

In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt

root_path = '/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/humanplate/results_35diam/'

overall_quant = []

for root, sub, files in os.walk(root_path):

   filenames = [os.path.join(root, filename) for filename in files 
                 if filename.startswith('combined')]
   
   for filename in filenames:
        print(os.path.join(root, filename))
        x = os.path.join(root, filename)
        y = x.split('/')[-2]
        df = pd.read_csv(filename)
        mean_combined = df['Mean'].mean()
       
        if mean_combined > 100: #you can change this value depending on the intensity of the far red channel
            result = 'Positive'
        else:
            result = 'Negative'
                
        overall_quant.append({'Subfolder': y, 'Data': mean_combined, 'Results': result})
        

oqdf = pd.DataFrame(overall_quant)
sorted_oqdf = oqdf.sort_values(by='Subfolder')
print(sorted_oqdf)
sorted_oqdf.to_csv('/Users/Alexander.Morano/Desktop/data/2024_12_19_GPCs_final_test/humanplate/results_35diam/finalresultsalpha.csv')